In [1]:
#Importing Libraries
import pandas as pd
import bs4
import requests
from tqdm import tqdm
import numpy as np
import datetime

In [4]:
#Main URL
base_url = "http://case.doe.gov.bd/index.php?option=com_content&view=category&id=8&Itemid=32&limitstart="

In [5]:
#each link has a pattern and a numerical value in it, hence making the numbers to create the primary links later

first = 0
last = 1920
inc = 30
page_numbers = []
page_numbers.extend([i for i in range(first, last+inc, inc)])

In [7]:
#going to each primary page and collecting the secondary links

link_list = []

for item in tqdm(page_numbers):
    extraction_url = base_url+str(item)
    
    r = requests.get(extraction_url)
    r.raise_for_status()
    
    page = bs4.BeautifulSoup(r.text, "html.parser")
    
    page_links = page.find_all('a')
    
    try:
        for li in page_links:
            if "Air Quality Index (AQI)_" in li.text:
                link_list.append("http://case.doe.gov.bd"+li.get('href'))
    except err:
          print(err)


 12%|██████████▏                                                                        | 8/65 [00:07<00:50,  1.13it/s]
Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\ProgramData\Anaconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [01:00<00:00,  1.07it/s]


In [8]:
#total number of links
len(link_list)

1917

In [9]:
#this function will collect date from a webpage 
def find_date(cur_page):
    divs = cur_page.find_all("span")
    for item in divs:
        temp = item.text
        if temp.startswith("Date"):
            return temp

In [10]:
len(link_list)

1917

In [11]:
#this function will collect the data from a table from a webpage, receives a url and send back the dataframe we need

def get_data_from_website(url):
    cols = ["Location","Air Quality Index (AQI)","AQI Category", "AQI Range","Date","URL"]
    df = pd.DataFrame(columns = cols)
    r = requests.get(url)
    r.raise_for_status()
    
    page = bs4.BeautifulSoup(r.text, "html.parser")
    
    for tr in page.find_all('tr'):
        tableDatas = tr.find_all("td")

        if len(tableDatas)>0:

            values = pd.Series(data = [item.text for item in tableDatas], index = cols[:-2])
            df.loc[len(df)] = values

    df['Date'] = find_date(page)
    df["URL"] = url
    
    return df

In [12]:
#making a list of dataframe from all the links 
dfs = []
for link in tqdm(link_list):
    dfs.append(get_data_from_website(link))

100%|██████████████████████████████████████████████████████████████████████████████| 1917/1917 [22:55<00:00,  1.39it/s]


In [106]:
DF = pd.DataFrame()
for df in dfs:
    DF = DF.append(df)

In [107]:
len(DF)

17352

In [108]:
DF.head()

,Location,Air Quality Index (AQI),AQI Category,AQI Range,Date,URL
0,LOCATION,Air Quality Index (AQI),AQI CATEGORY,AQI RANGE,Date: 26/06/2019,http://case.doe.gov.bd/index.php?option=com_co...
1,DHAKAb,122,CAUTION,,Date: 26/06/2019,http://case.doe.gov.bd/index.php?option=com_co...
2,GAZIPURc,39,GOOD,,Date: 26/06/2019,http://case.doe.gov.bd/index.php?option=com_co...
3,NARAYANGANJc,53,MODERATE,,Date: 26/06/2019,http://case.doe.gov.bd/index.php?option=com_co...
4,CHITTAGONGc,46,GOOD,,Date: 26/06/2019,http://case.doe.gov.bd/index.php?option=com_co...


In [109]:
DF.Location.unique()

array(['LOCATION', 'DHAKAb', 'GAZIPURc', 'NARAYANGANJc', 'CHITTAGONGc',
       'SYLHETc', 'KHULNAc', 'RAJSHAHIc', 'BARISHALc', 'SAVARc',
       'MYMENSINGHc', 'RANGPURc', 'CUMILLAc', '', 'Dhakab', 'Chittagongc',
       'Gazipurc', 'Narayanganjc', 'Sylhetc', 'Khulnac', 'Rajshahic',
       'Barisalc', 'Dhakac', 'DhakaC', 'Dhakaa', 'Chittagongb',
       'ChittagongC', 'Sylhetc2', 'gongb', 'Sylhetcc'], dtype=object)

In [110]:
DF.drop(DF[DF.Location.isin(['LOCATION','','gongb'])].index,inplace=True)
DF.reset_index(drop=True,inplace=True)

In [111]:
len(DF)

13480

In [112]:
DF.columns = [col.lower().replace(" ","_") for col in DF.columns]

In [113]:
DF.location.unique()

array(['DHAKAb', 'NARAYANGANJc', 'CHITTAGONGc', 'SYLHETc', 'KHULNAc',
       'RAJSHAHIc', 'BARISHALc', 'MYMENSINGHc', 'RANGPURc', 'CUMILLAc',
       'Dhakab', 'Gazipurc', 'Narayanganjc', 'Sylhetc', 'Khulnac',
       'Rajshahic', 'Barisalc', 'Dhakac', 'DhakaC', 'Dhakaa', 'Sylhetc2',
       'Chittagongb', 'Sylhetcc'], dtype=object)

In [114]:
loc_dict = {'DHAKAb':"dhaka", 'NARAYANGANJc':"narayanganj", 'CHITTAGONGc':"chittagong", 'SYLHETc':"sylhet", 'KHULNAc':"khulna",
            'RAJSHAHIc':"rajshahi", 'BARISHALc':"barishal", 'MYMENSINGHc':"mymensingh", 'RANGPURc':"rangpur", 
            'CUMILLAc':"cumilla",'Dhakab':"dhaka", 'Gazipurc':"gazipur", 'Narayanganjc':"narayanganj", 'Sylhetc':"sylhet", 
            'Khulnac':"khulna",'Rajshahic':"rajshahi", 'Barisalc':"barishal", 'Dhakac':"dhaka", 'DhakaC':"dhaka", 
            'Dhakaa':"dhaka", 'Sylhetc2':"sylhet",'Chittagongb':"chittagong", 'Sylhetcc':"sylhet"}

In [115]:
DF.location = DF.location.map(loc_dict)

In [116]:
DF["air_quality_index_(aqi)"] = DF["air_quality_index_(aqi)"].str.replace("\xa0 ","")

In [117]:
DF["air_quality_index_(aqi)"] = DF["air_quality_index_(aqi)"].str.replace("\xa0","")

In [118]:
DF["air_quality_index_(aqi)"] = DF["air_quality_index_(aqi)"].str.replace(" ","")

In [119]:
DF["air_quality_index_(aqi)"] = DF["air_quality_index_(aqi)"].str.replace('DATANOTAVAILABLE',"DNA")

In [120]:
DF["air_quality_index_(aqi)"] = DF["air_quality_index_(aqi)"].str.replace("DNA*","DNA")

In [121]:
DF["air_quality_index_(aqi)"].unique()

array(['122', '53', '46', '40', '26', '66', '23', '30', '98', '87', '64',
       '51', '28', 'DNA', '77', '37', '135', '111', '93', '63', '52', '56',
       '34', '117', '89', '49', '25', '69', '41', '114', '68', '35', '18',
       '42', '27', '88', '20', '14', '113', '86', '33', '29', '36', '15',
       '139', '108', '38', '44', '79', '162', '209', '47', '82', '50',
       '78', '74', '148', '142', '58', '39', '16', '65', '83', '54', '144',
       '71', '115', '85', '128', '102', '57', '131', '103', '92', '45',
       '112', '70', '97', '110', '100', '76', '48', '19', '22', '67', '55',
       '21', '31', '62', '32', '24', '59', '72', '73', '133', '90', '81',
       '105', '109', '104', '125', '174', '149', '43', '84', '126', '75',
       '10', '124', '119', '95', '120', '101', '136', '168', '118', '210',
       '227', '130', '106', '146', '141', '123', '147', '153', '178', '94',
       '80', '91', '172', '129', '116', '189', '134', '99', '223', '132',
       '214', '152', '150', '220'

In [122]:
DF["air_quality_index_(aqi)"] = DF["air_quality_index_(aqi)"].str.lower()

In [123]:
DF['aqi_category'] = DF['aqi_category'].str.replace("\xa0 ","")

In [124]:
DF['aqi_category'] = DF['aqi_category'].str.replace("\xa0","")

In [125]:
DF['aqi_category'] = DF['aqi_category'].str.replace("\n","")

In [126]:
DF['aqi_category'] = DF['aqi_category'].str.lower()

In [127]:
DF['aqi_category'] = DF['aqi_category'].str.strip()

In [128]:
DF['aqi_category'] = DF['aqi_category'].str.replace("ââ"," ")

In [129]:
DF['aqi_category'] = DF['aqi_category'].str.replace("â"," ")

In [130]:
DF['aqi_category'] = DF['aqi_category'].str.replace('extremelyunhealthy','extremely unhealthy')

In [131]:
DF['aqi_category'] = DF['aqi_category'].str.replace( 'exetremelyunhealthy','extremely unhealthy')

In [140]:
DF['aqi_category'] = DF['aqi_category'].str.replace('exetremely unhealthy','extremely unhealthy')

In [144]:
DF['aqi_category'] = DF['aqi_category'].str.replace('extremelyunhealthy','extremely unhealthy')

In [132]:
DF['aqi_category'] = DF['aqi_category'].str.replace('extremly unhealthy','extremely unhealthy')

In [133]:
DF['aqi_category'] = DF['aqi_category'].str.replace('moderatre','moderate')

In [134]:
DF['aqi_category'] = DF['aqi_category'].str.replace('modetate','moderate')

In [135]:
DF['aqi_category'] = DF['aqi_category'].str.replace('mderate','moderate')

In [136]:
DF['aqi_category'] = DF['aqi_category'].str.replace( 'veary unhealthy','very unhealthy')

In [138]:
DF['aqi_category'] = DF['aqi_category'].str.replace( 'veryunhealthy','very unhealthy')

In [146]:
DF['aqi_category'] = DF['aqi_category'].str.replace( 'veryunhealthy','very unhealthy')

In [142]:
DF['aqi_category'] = DF['aqi_category'].str.replace( ' unhealthy','unhealthy')

In [148]:
DF['aqi_category'] = DF['aqi_category'].str.replace( 'data not available','dna')

In [156]:
DF['aqi_category'].unique()

array(['caution', 'moderate', 'good', 'dna', 'unhealthy', 'very unhealthy',
       'extremely unhealthy', nan, 'extremely very unhealthy'], dtype=object)

In [155]:
DF.ix[DF[DF['aqi_category'].isin([''])].index,'aqi_category'] = np.nan

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [160]:
DF.ix[DF[DF["aqi_range"].isin(['','ÂÂ\xa0ÂÂ\xa0','\xa0\xa0',])].index,"aqi_range"] = np.nan

In [164]:
DF['date'] = DF['date'].str.replace( 'Date: ','')

In [167]:
DF['date'] = DF['date'].str.replace( 'Date:','')

In [181]:
DF['date'] = DF['date'].str.replace( "\xa0",'')

In [185]:
DF['date'] = DF['date'].str.replace( "201557",'2015')

In [186]:
DF.date = pd.Series([datetime.datetime.strptime(item,"%d/%m/%Y").date() if pd.notnull(item) else item for item in DF.date],
                    index=DF.index)

In [188]:
DF.head()

,location,air_quality_index_(aqi),aqi_category,aqi_range,date,url
0,dhaka,122,caution,NaN,2019-06-26,http://case.doe.gov.bd/index.php?option=com_co...
1,narayanganj,53,moderate,NaN,2019-06-26,http://case.doe.gov.bd/index.php?option=com_co...
2,chittagong,46,good,NaN,2019-06-26,http://case.doe.gov.bd/index.php?option=com_co...
3,sylhet,40,good,NaN,2019-06-26,http://case.doe.gov.bd/index.php?option=com_co...
4,khulna,26,good,NaN,2019-06-26,http://case.doe.gov.bd/index.php?option=com_co...


In [189]:
DF.to_csv("AQI_Data.csv",encoding="utf-8",index=False)

In [8]:
DF.to_stata("AQI_Data_30Jun2019.dta")

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\stata.py:2070: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    b'air_quality_index_(aqi)'   ->   air_quality_index__aqi_

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  warnings.warn(ws, InvalidColumnName)
